In [45]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate,FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector



chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
{
    "country": "France",
    "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Croissant and Baguette
Currency: Euro
"""
    },
    {
        "country": "Japan",
        "answer": """
Here is what I know:
Capital: Tokyo
Language: Japanese
Food: Sushi and Ramen
Currency: Yen
"""
    },
    {
        "country": "Italy",
        "answer": """
Here is what I know:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
"""
    },
    {
        "country": "Korea",
        "answer": """
Here is what I know:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: Won
"""
    },
    {
        "country": "Spain",
        "answer": """
Here is what I know:
Capital: Madrid
Language: Spanish
Food: Paella and Tapas
Currency: Euro
"""
    }
]


class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]
    
    def add_example(self, example):
        self.examples.append(example)
    



example_prompt = PromptTemplate.from_template("Human: {country}\nAI:{answer}")
print(example_prompt)
print("--------------------------")

example_selector = RandomExampleSelector(
    examples=examples,
)
print(example_selector)
print("--------------------------")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}",
    input_variables=["country"],
)
print(prompt)
print("--------------------------")

prompt.format(country="Brazil")

input_variables=['answer', 'country'] template='Human: {country}\nAI:{answer}'
--------------------------
--------------------------
input_variables=['country'] example_selector=<__main__.RandomExampleSelector object at 0x12ff32ed0> example_prompt=PromptTemplate(input_variables=['answer', 'country'], template='Human: {country}\nAI:{answer}') suffix='Human: What do you know about {country}'
--------------------------


'Human: Spain\nAI:\nHere is what I know:\nCapital: Madrid\nLanguage: Spanish\nFood: Paella and Tapas\nCurrency: Euro\n\n\nHuman: What do you know about Brazil'

In [49]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

prompt.format(country="Germany")

'What is the capital of Germany'

In [54]:
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)


Arrr matey! Me favorite grub be a hearty plate of salted beef and hardtack! Aye, nothing beats the taste of the sea on me tongue! Arrr!

AIMessageChunk(content='Arrr matey! Me favorite grub be a hearty plate of salted beef and hardtack! Aye, nothing beats the taste of the sea on me tongue! Arrr!')